In [1]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [34]:
def Evaluate(self, Light):
    s = self.data[:, Light]
    s_re = self.reconstructed_spectrum[:, Light]
    MAE = np.mean(np.absolute(s - s_re))
    RMSE = np.sqrt(np.mean(np.square(s - s_re)))
    RRMSE = RMSE / np.mean(s_re)

    # GFC
    num = abs(np.sum(s * s_re))
    denom_s = np.sqrt(np.sum(s ** 2) + 1e-9)
    denom_s_re = np.sqrt(np.sum(s_re ** 2) + 1e-9)
    GFC = num / (denom_s * denom_s_re)

    print("MAE: ", MAE)
    print("RMSE: ", RMSE)
    print("RRMES: ", RRMSE)
    print("GFC: ", GFC)


def Plot(self, Light):
    # 创建一个包含三个子图的图像
    fig, axs = plt.subplots(3, 1)
    s = self.data[:, Light]
    r = self.reconstructed_spectrum[:, Light]

    # 绘制真实光谱图
    axs[0].plot(self.wavelengths, s, label='True Spectrum')
    axs[0].set_xlabel('Wavelength (nm)')
    axs[0].set_ylabel('Intensity')
    axs[0].set_title('True Spectrum vs Reconstructed Spectrum')
    axs[0].legend()

    # 绘制重建光谱图
    axs[1].plot(self.wavelengths, r, label='Reconstructed Spectrum')
    axs[1].set_xlabel('Wavelength (nm)')
    axs[1].set_ylabel('Intensity')
    axs[1].legend()

    # 绘制重建光谱和真实光谱叠加的图
    axs[2].plot(self.wavelengths, s, label='True Spectrum')
    axs[2].plot(self.wavelengths, r, label='Reconstructed Spectrum')
    axs[2].set_xlabel('Wavelength (nm)')
    axs[2].set_ylabel('Intensity')
    axs[2].legend()

    plt.tight_layout()
    plt.show()

In [43]:
class SPDPCAandCluster:
    def __init__(self, spd_data, n_components, n_clusters):
        self.data = spd_data
        self.n_components = n_components
        self.n_clusters = n_clusters
        self._pca = PCA(n_components=self.n_components)
        self._kmeans = KMeans(n_clusters=self.n_clusters, random_state=0)

    def cluster_data(self):
        self.labels = self._kmeans.fit_predict(self.data.T)
        self.clustered_data = [self.data[:, self.labels == i] for i in range(self.n_clusters)]

    def fit(self):
        self.cluster_data()
        self.pca_models = {}

        for i, cluster in enumerate(self.clustered_data):
            pca = PCA(n_components=self.n_components)
            pca.fit(cluster.T)
            self.pca_models[i] = pca

    def reconstruct(self):
        reconstructed_data = np.zeros_like(self.data)

        for i, cluster in enumerate(self.clustered_data):
            pca_model = self.pca_models[i]
            scores = pca_model.transform(cluster.T)
            reconstructed_cluster = pca_model.inverse_transform(scores).T
            reconstructed_data[:, self.labels == i] = reconstructed_cluster

        return reconstructed_data






In [5]:
spd = np.load("/Users/dongtianchi/Documents/GIT/SpectralReconstruction/ComputationalSpectrometers/NNLS/cleandata.npy", allow_pickle= True)

In [44]:
test = SPDPCAanfCluster(spd,8,2)
test.cluster_data()
test.fit()

/opt/anaconda3/envs/ForEXE/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/anaconda3/envs/ForEXE/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


UFuncTypeError: Cannot cast ufunc 'svd_m_s' input from dtype('O') to dtype('float64') with casting rule 'same_kind'

In [15]:
S1 = spd[:,0]
S3 = spd[:,2]
s500 = spd[:,499]
S700 = spd[:,699]
S1000 = spd[:,999]
S1494 = spd[:,-1]


In [52]:
class SPDReClusterPCA:
    def __init__(self, spd_data, n_components, n_clusters):
        self.data = spd_data
        self.n_components = n_components
        self.n_clusters = n_clusters
        self._pca = PCA(n_components=self.n_components)
        self._kmeans = KMeans(n_clusters=self.n_clusters, random_state=0)
        self.wavelengths = np.arange(360, 831)
        self.mu = np.array([415, 445, 480, 515, 555, 590, 630, 680])
        self.sigma = np.array([11.0403, 12.7388, 15.2866, 16.5605, 16.5605, 16.9851, 21.2314, 22.0807])

    def Reconstructed_spectrum(self):
        labels = self._kmeans.fit_predict(self.data.T)
        self.clustered_data = [self.data[:, labels == i] for i in range(self.n_clusters)]
        reconstructed_data = np.zeros_like(self.data)

        for i, cluster in enumerate(self.clustered_data):
            scores = self._pca.fit_transform(cluster.T)
            filters = np.zeros((len(self.mu), len(self.wavelengths)))
            for j in range(8):
                filters[j] = np.exp(-0.5 * ((self.wavelengths - self.mu[j]) / self.sigma[j])**2)
            M = np.dot(filters, scores.T)
            response_matrix = np.dot(filters, cluster)
            mean_response = np.mean(response_matrix, axis=1)
            mean_response = mean_response.reshape(-1, 1)
            M_inv = np.linalg.pinv(M)
            response_diff = response_matrix - mean_response
            a_hat = np.dot(M_inv, response_diff)
            mean_spd = np.mean(cluster, axis=1)
            reconstructed_spectrum = np.dot(scores, a_hat) + mean_spd[:, np.newaxis]
            self.reconstructed_data[:, labels == i] = reconstructed_spectrum

        

        
    
    def Plot(self, Light):
        # 创建一个包含三个子图的图像
        fig, axs = plt.subplots(3, 1)
        s = self.data[:,Light]
        r = self.reconstructed_data[:,Light]

        # 绘制真实光谱图
        axs[0].plot(self.wavelengths, s, label='True Spectrum')
        axs[0].set_xlabel('Wavelength (nm)')
        axs[0].set_ylabel('Intensity')
        axs[0].set_title('True Spectrum vs Reconstructed Spectrum')
        axs[0].legend()

        # 绘制重建光谱图
        axs[1].plot(self.wavelengths, r, label='Reconstructed Spectrum')
        axs[1].set_xlabel('Wavelength (nm)')
        axs[1].set_ylabel('Intensity')
        axs[1].legend()

        # 绘制重建光谱和真实光谱叠加的图
        axs[2].plot(self.wavelengths, s, label='True Spectrum')
        axs[2].plot(self.wavelengths, r, label='Reconstructed Spectrum')
        axs[2].set_xlabel('Wavelength (nm)')
        axs[2].set_ylabel('Intensity')
        axs[2].legend()

        plt.tight_layout()
        plt.show()

    def Evaluate(self, light):
        s = self.data[:, light]
        s_re = self.reconstructed_spectrum[:,light]
        MAE = np.mean(np.absolute(s - s_re))
        RMSE = np.sqrt(np.mean(np.square(s- s_re)))
        RRMSE = RMSE / np.mean(s_re)

        #GFC
        num = abs(np.sum(s * s_re))
        denom_s = np.sqrt(np.sum(s ** 2) + 1e-9)
        denom_s_re = np.sqrt(np.sum(s_re ** 2) + 1e-9)
        GFC =num / (denom_s * denom_s_re)

        print("MAE: ", MAE)
        print("RMSE: ", RMSE)
        print("RRMES: ", RRMSE)
        print("GFC: ", GFC)


In [53]:
test = SPDReClusterPCA(spd, 8, 2)
test.Reconstructed_spectrum()

/opt/anaconda3/envs/ForEXE/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


ValueError: shapes (8,471) and (8,904) not aligned: 471 (dim 1) != 8 (dim 0)